In [1]:
from importlib import reload
import sys

src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

max_length = 128

train = True
debias = True

In [2]:
import data_generator;reload(data_generator)

train_data, val_data, test_data = data_generator.GetData(max_length, sample=10000)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
100%|██████████| 30000/30000 [00:08<00:00, 3354.24it/s]


        tag  cat  occurences
0    B-MISC    0          58
1     I-LOC    1       25185
2    I-MISC    2       10213
3     I-ORG    3       18042
4     I-PER    4       83183
5         O    5      520930
6  [nerCLS]    6       30000
7  [nerPAD]    7     2960644
8  [nerSEP]    8       30000
9    [nerX]    9      161745

                tag  cat  occurences
0  AFRICAN-AMERICAN    0       14932
1          EUROPEAN    1       15068
2         [raceCLS]    2       30000
3         [racePAD]    3     2960644
4         [raceSEP]    4       30000
5           [raceX]    5      789356

           tag  cat  occurences
0       FEMALE    0       15019
1         MALE    1       14981
2  [genderCLS]    2       30000
3  [genderPAD]    3     2960644
4  [genderSEP]    4       30000
5    [genderX]    5      789356



In [3]:
import model_utils; reload(model_utils)

model = model_utils.NER(max_length)
    
model.generate(bert_train_layers=4)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [ ]:
if train:
    model.fit(
        train_data,
        val_data,
        epochs=8,
        batch_size=32,
        debias=debias
    ) 


val_acc_ner: 0.98; val_acc_ner_non_other: 0.93;  val_acc_gender: 0.50; val_acc_race: 0.50



val_acc_ner: 0.99; val_acc_ner_non_other: 0.95;  val_acc_gender: 0.50; val_acc_race: 0.50



val_acc_ner: 0.99; val_acc_ner_non_other: 0.96;  val_acc_gender: 0.50; val_acc_race: 0.50



val_acc_ner: 0.99; val_acc_ner_non_other: 0.96;  val_acc_gender: 0.50; val_acc_race: 0.50



val_acc_ner: 0.99; val_acc_ner_non_other: 0.96;  val_acc_gender: 0.50; val_acc_race: 0.50



val_acc_ner: 0.99; val_acc_ner_non_other: 0.97;  val_acc_gender: 0.50; val_acc_race: 0.50



val_acc_ner: 0.99; val_acc_ner_non_other: 0.96;  val_acc_gender: 0.50; val_acc_race: 0.50


In [ ]:
model.score(test_data)

In [ ]:
bias = model.getBiasedPValues(test_data, num_iterations=10000)

In [ ]:
bias